In [7]:
from poem.instance_creation_factories.triples_factory import TriplesFactory
from poem.models.unimodal.ermlp import ERMLP
from torch import optim
import numpy as np
from poem.training.owa import OWATrainingLoop
import os
import logging
import sys
import timeit
from torch import nn
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stderr)

**Step 1: Create instances**

In [5]:
path_to_training_data = '../tests/resources/test.txt'

In [6]:
factory = TriplesFactory(path=path_to_training_data)

**Step 2: Configure KGE model**

In [8]:
kge_model = ERMLP(
    triples_factory = factory,
    embedding_dim = 50,
    criterion = nn.MarginRankingLoss(margin=1., reduction='mean'),
    random_seed = 2, 
    )

**Step 3: Configure Training Loop**

In [12]:
optimizer = optim.Adam(params=kge_model.get_grad_params())
owa_training_loop = OWATrainingLoop(model=kge_model, optimizer=optimizer)

**Step 4: Train**

In [13]:
losses = owa_training_loop.train(
    num_epochs=5,
    batch_size=256
)


Training epoch on cpu:   0%|          | 0/5 [00:00<?, ?it/s]

RuntimeError: Number of dimensions of repeat dims can not be smaller than number of dimensions of tensor